# IMP-Logics DEMO

This is the demo of IMP-Logics, [a freely available java library](https://github.com/inLabFIB/imp-logics), implementing the metamodels of Datalog schemas, and Dependency schemas (i.e., Datalog+/-)

Full demo code can be found [here](https://github.com/inLabFIB/imp-logics-demo/)

## Loading Project Dependencies
Please, run this scripts to load all the dependencies and auxiliar functions to run the demo

In [75]:
%%loadFromPOM
<dependencies>
    <dependency>
        <groupId>org.antlr</groupId>
        <artifactId>antlr4-runtime</artifactId>
        <version>4.12.0</version>
    </dependency>
    <!-- ASSERTJ - for testings and asserts -->
    <dependency>
        <groupId>org.assertj</groupId>
        <artifactId>assertj-core</artifactId>
        <version>3.24.2</version>
        <scope>test</scope>
    </dependency>
</dependencies>

In [76]:
%maven org.assertj:assertj-core:3.24.2
%jars imp-logics-2.0.0.jar
%jars imp-logics-2.0.0-tests.jar
%jars ontological-queries-rewriting-1.0-SNAPSHOT.jar

import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.domain.DependencySchema;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.domain.TGD;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.domain.EGD;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.analyzers.DatalogPlusMinusAnalyzer;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.analyzers.egds.NonConflictingEGDsAnalyzer;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.parser.DependencySchemaParser;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.printer.DependencySchemaPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.assertions.LogicSchemaAssertions;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.domain.*;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.parser.LogicSchemaWithIDsParser;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.parser.QueryParser;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.printer.LogicSchemaPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.printer.QueryPrinter;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.EqualityReplacer;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.LogicProcessPipeline;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.SchemaUnfolder;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.processes.SingleDerivationRuleTransformer;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.OBDAMapping;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.Rewriter;
import edu.upc.fib.inlab.imp.kse.ontological_queries_rewriting.utils.normalizers.TGDNormalizerProcess;
import edu.upc.fib.inlab.imp.kse.logics.dependencyschema.services.processes.*;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.services.parser.QueryParser;
import edu.upc.fib.inlab.imp.kse.logics.logicschema.assertions.QueryAssert;


import java.util.Collection;
import java.util.HashSet;
import java.util.List;
import java.util.Set;

import static org.assertj.core.api.Assertions.assertThat;

In [77]:
private static void printWithHeader(String logicConstraintUsedVariables, String content) {
    System.out.println("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
    System.out.println(content);
}

private static void printWithHeaderInline(String logicConstraintUsedVariables, String content) {
    System.out.print("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
    System.out.println(content);
}
private static void printHeaderInline(String logicConstraintUsedVariables) {
    System.out.print("\u001B[1m" + logicConstraintUsedVariables + ":\033[0m ");
}

## DEMO START

We structure the Demo in three parts:
1. Showing the capabilities of the Datalog metamodel
2. Showing the capabilities of the Dependency metamodel (i.e., Datalog+/-)
3. Showing how to use both to implement and validate, for instance, an OBDA query rewritting

### PART 1: LogicSchema, the Datalog metamodel

#### Parsing a LogicSchema
We start by parsing a logic schema regarding some university. That is, the Teachers and Students it has, and the AcademicRecords it stores.

![title](./DB_Demo.png)

In particular, such schema will contain the following predicates:

In [52]:
Set<Predicate> predicates = Set.of(
    new Predicate("DB_AcademicRecord", 3),
    new Predicate("DB_Student", 2),
    new Predicate("DB_Studies", 2),
    new Predicate("DB_Subject", 1),
    new Predicate("DB_AssistantTeacher", 2),
    new Predicate("DB_TenuredTeacher", 2),
    new Predicate("DB_Teaches", 2),
    new Predicate("DB_ComposesPlan", 2),
    new Predicate("DB_StudiesPlan", 1),
    new Predicate("DB_PublishesAbout", 3)
);

Assume that we want to define 3 constraints (written as denial constraints -aka negative constraints-), and some derived predicates.
The easiest way is by parsing them.

Let's first define the constraints and derivation rules as Strings:

In [53]:
String logicSchemaString = """
    %% Schema Logic Constraints
    % AcademicRecord reference key to Student
    @AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))
    IsStudent(studentName) :- DB_Student(studentName, age)
                  
    % Teacher must be over 18
    @TeachersMustBeOver18 :- Teacher_view(name, age), age < 18
    Teacher_view(name, age) :- DB_AssistantTeacher(name, age)
    Teacher_view(name, age) :- DB_TenuredTeacher(name, age)
                    
    % A teacher cannot teach himself
    @TeacherCannotTeachHimself :- DB_Teaches(teacherName, subject), DB_Studies(studentName, subject), teacherName=studentName
    """;

Now we can parse them into a LogicSchema object.

In [54]:
LogicSchemaWithIDsParser logicSchemaParser = new LogicSchemaWithIDsParser();
LogicSchema logicSchema = logicSchemaParser.parse(logicSchemaString, predicates);

We can print the schema:

In [55]:
LogicSchemaPrinter logicSchemaPrinter = new LogicSchemaPrinter();
printWithHeader("Logic Schema", logicSchemaPrinter.print(logicSchema));

Logic Schema: 
@TeacherCannotTeachHimself :- DB_Teaches(teacherName, subject), DB_Studies(studentName, subject), teacherName=studentName
@AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))
@TeachersMustBeOver18 :- Teacher_view(name, age), age<18
IsStudent(studentName) :- DB_Student(studentName, age)
Teacher_view(name, age) :- DB_AssistantTeacher(name, age)
Teacher_view(name, age) :- DB_TenuredTeacher(name, age)



What else can we do?

#### LogicSchema navigation

To show the navigation capabilities, we will pick the logic constraint `@AcademicRecordFKToStudent`, and from there, we will start visiting its literals, variables, predicates, etc.

In [56]:
LogicConstraint selectedConstraint = logicSchema.getLogicConstraintByID(new ConstraintID("AcademicRecordFKToStudent"));
printWithHeaderInline("Selected Logic Constraint", logicSchemaPrinter.visit(selectedConstraint));

Selected Logic Constraint: @AcademicRecordFKToStudent :- DB_AcademicRecord(studentName, subject, eval), not(IsStudent(studentName))


We can check the used variables in the constraint body

In [57]:
Set<Variable> usedVariables = selectedConstraint.getBody().getUsedVariables();
printHeaderInline("Used Variables in Body");
for (Variable v : usedVariables) System.out.print(v.getName() + " ");

Used Variables in Body: studentName subject eval 

We can also navigate to its literals, predicates, and derivation rules it depends on. You can see the navegability possibilities in the following image:

![title](./DatalogMetamodel.png)

We can select a literal of the constraint, and check its positive/negative polarity, whether it is ground or not, or if it is base or derived.

In [58]:
OrdinaryLiteral olit = (OrdinaryLiteral) selectedConstraint.getBody().get(1);
printWithHeaderInline("Selected Ordinary Literal", olit.toString());

System.out.println("Ordinary Literal is negative: " + olit.isNegative());
System.out.println("Ordinary Literal is ground: " + olit.isGround());
System.out.println("Ordinary Literal is base: " + olit.isBase());

Selected Ordinary Literal: not(IsStudent(studentName))
Ordinary Literal is negative: true
Ordinary Literal is ground: false
Ordinary Literal is base: false


The predicate of an ordinary literal can be obtained, and we can check whether it is base or derived

In [17]:
Predicate olitPredicate = olit.getPredicate();

From a derived predicate we can access its definition rules

In [ ]:
List<DerivationRule> derivationRules = olitPredicate.getDerivationRules();
printHeaderInline("Predicate's derivation rules:");
for (DerivationRule dr : derivationRules) System.out.print(logicSchemaPrinter.visit(dr));

#### LogicSchema operations

We refer as operations to those methods already available in the main metamodel classes.
For instance, given an atom we can unfold it:

In [59]:
Predicate isStudentPredicate = logicSchema.getPredicateByName("IsStudent");
Atom johnAtom = new Atom(isStudentPredicate, List.of(new Constant("John")));
printWithHeaderInline("Original atom", johnAtom.toString());
printWithHeaderInline("Derivation rules it has", johnAtom.getPredicate().getDerivationRules().toString());
printWithHeaderInline("Atom after unfolding", johnAtom.unfold().toString());

Original atom: IsStudent(John)
Derivation rules it has: [IsStudent(studentName) :- DB_Student(studentName, age)]
Atom after unfolding: [DB_Student(John, age)]


The unfold is also available for list of literals, and it takes care of avoiding variable name clashing:

In [60]:
OrdinaryLiteral johnStudent = new OrdinaryLiteral(johnAtom);
OrdinaryLiteral maryStudent = new OrdinaryLiteral(new Atom(isStudentPredicate, List.of(new Constant("Mary"))));
ImmutableLiteralsList literalsList = new ImmutableLiteralsList(List.of(johnStudent, maryStudent));
printWithHeaderInline("Original literalsList", literalsList.toString());
printWithHeaderInline("Unfolding the second literal", literalsList.unfold(1).toString());
printWithHeaderInline("Unfolding both literals", literalsList.unfold(1).get(0).unfold(0).toString());

Original literalsList: IsStudent(John), IsStudent(Mary)
Unfolding the second literal: [IsStudent(John), DB_Student(Mary, age)]
Unfolding both literals: [DB_Student(John, age'), DB_Student(Mary, age)]


Do note that the unfolding has avoided a variable name clash with age.

#### LogicSchema services

We refer as services to those operations that are not inside the main class diagram.

Just for example, we will show some transformation services. Transformation services receives as input a logic schema and outputs a new logic schema
after applying some transformation into it. Such processes can be executed in a pipeline.

For our demo, we will use the `EqualityReplacer`, `SchemaUnfolder` processes.

In [ ]:
LogicProcessPipeline pipeline = new LogicProcessPipeline(List.of(
        new EqualityReplacer(),
        new SchemaUnfolder(false)
));
LogicSchema modifiedLogicSchema = pipeline.execute(logicSchema);
printWithHeader("Modified schema", logicSchemaPrinter.print(modifiedLogicSchema));

### PART 2: DependencySchema, the Datalog+/- metamodel

#### Parsing a DependencySchema

We will now parse an ontology over the same university domain.

In [61]:
String dependencySchemaString = """
    % If a student passes a subject, the student has some evaluation
    HasPassed(student, subject) -> Exam(teacher, student, subject, data)
                    
    % If a teacher teaches a subject a student is coursing, the teacher evaluates the student
    Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
                    
    % If a teacher is expert in a subject from a study plan, the teacher gives the subject
    ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)
                    
    % A subject has, at most, one exam per day
    Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> teacher = teacher2
    Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> student = student2
    """;
DependencySchemaParser dependencySchemaParser = new DependencySchemaParser();
DependencySchema dependencySchema = dependencySchemaParser.parse(dependencySchemaString);

We can, for instance, print the dependency schema

In [62]:
DependencySchemaPrinter dependencySchemaPrinter = new DependencySchemaPrinter();
printWithHeader("Dependency Schema", dependencySchemaPrinter.print(dependencySchema));

Dependency Schema: 
HasPassed(student, subject) -> Exam(teacher, student, subject, data)
Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)
Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> teacher=teacher2
Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> student=student2



Let's see what else can we do

#### DependencySchema navigation

![title](./DependencyMetamodel.png)

We can pick the TGDs and EGDs of the schema, and similarly as before, navigate through the metamodel.

In [63]:
TGD tgd = dependencySchema.getAllTGDs().get(0);
printWithHeaderInline("Selected TGD", tgd.toString());

EGD egd = dependencySchema.getAllEGDs().get(0);
printWithHeaderInline("Selected EGD", egd.toString());

EqualityComparisonBuiltInLiteral equality = egd.getHead();
printWithHeaderInline("Selected equality", equality.toString());

Selected TGD: HasPassed(student, subject) -> Exam(teacher, student, subject, data)
Selected EGD: Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> teacher = teacher2
Selected equality: teacher = teacher2


#### DependencySchema operations

For instance, we can check whether the previous TGD is linear, or guarded.

In [64]:
System.out.println("TGD is linear: " + tgd.isLinear());
System.out.println("TGD is guarded: " + tgd.isGuarded());

TGD is linear: true
TGD is guarded: true


#### DependencySchema services
Among other services, we can check whether the EGDs are conflicting with the TGDs:

In [65]:
NonConflictingEGDsAnalyzer nonConflictingEGDsAnalyzer = new NonConflictingEGDsAnalyzer();
boolean separable = nonConflictingEGDsAnalyzer.areEGDsNonConflictingWithTGDs(dependencySchema);
System.out.print("EGDs of schema are non conflicting / separable: " + separable);

EGDs of schema are non conflicting / separable: true

We will now analyze which Datalog+/- languages this dependency schema satisfies

In [66]:
DatalogPlusMinusAnalyzer analyzer = new DatalogPlusMinusAnalyzer();
Set<DatalogPlusMinusAnalyzer.DatalogPlusMinusLanguage> languages = analyzer.getDatalogPlusMinusLanguages(dependencySchema);
printHeaderInline("This dependency schema is: ");
for (DatalogPlusMinusAnalyzer.DatalogPlusMinusLanguage dl : languages) System.out.print(dl.name() + " ");

This dependency schema is: : WEAKLY_GUARDED STICKY 

There are also some transformation services, but we will take them a look on the thirt part of the demonstration.

### PART 3: Using IMP-Logics for OBDA

We will show how IMP-Logics can be used to implement OBDA concepts such as an ontology query-rewritting.
We will assume that:
- Our logicSchema is a relational database
- Our dependencySchema is an ontology defined on top of the previous database

We start by "normalizing" the dependencySchema. That is, we need to obtain a new dependencySchema where each TGD head has at most one atom with at most one existential variable.

We can easily implement such normalization by concatenating two DependencySchema services from IMP-Logics

In [67]:
DependencyProcessPipeline dependencyProcessPipeline = new DependencyProcessPipeline(List.of(
                new SingleHeadTGDTransformer(),                      //provided by IMP-Logics
                new SingleExistentialVarTGDTransformer()));          //provided by IMP-Logics
DependencySchema normalizedDependencySchema = dependencyProcessPipeline.execute(dependencySchema);

DependencySchemaPrinter dependencySchemaPrinter = new DependencySchemaPrinter();
printWithHeader("Normalized Dependency Schema", dependencySchemaPrinter.print(normalizedDependencySchema));

Normalized Dependency Schema: 
HasPassed(student, subject) -> AUX1(student, subject, teacher)
AUX1(student, subject, teacher) -> AUX2(student, subject, teacher, data)
AUX2(student, subject, teacher, data) -> Exam(teacher, student, subject, data)
Teaches(teacher, subject), Studies(student, subject) -> Exam(teacher, student, subject, data)
ExpertIn(teacher, subject), ComposesPlan(subject, studyPlan) -> Teaches(teacher, subject)
Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> teacher=teacher2
Exam(teacher, student, subject, data), Exam(teacher2, student2, subject, data) -> student=student2



We now define some mappings from the predicates of the dependencySchema (the ontology) to the predicates of the logicSchema (the database).
To do so, we reuse the Query class of IMP-Logics, and define our new class OBDAMapping.

In [68]:
String mappingDBQueriesString = """
    % HasPassed(student, subject)
    (student, subject) :- DB_AcademicRecord(student, subject, mark), mark > 5
    
    % Exam(teacher, student, subject, data)
    (teacher, student, subject, data) :- DB_Exam(teacher, student, subject, data)
    
    % Teaches(teacher, subject)
    (teacher, subject) :- DB_Teaches(teacher, subject)
    
    % Studies(student, subject)
    (student, subject) :- DB_Studies(student, subject)
    
    % ExpertIn(teacher, subject)
    (teacher, subject) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2
    
    % ComposesPlan(subject, studyPlan)
    (subject, studyPlan) :- DB_ComposesPlan(subject, studyPlan)
    """;
QueryParser queryParser = new QueryParser();                    //Provided by IMP-Logics
List<Query> mappingDBQueries = queryParser.parse(mappingDBQueriesString, predicates);
OBDAMapping mapping = new OBDAMapping.OBDAMappingBuilder()
    .addMapping(normalizedDependencySchema.getPredicateByName("HasPassed"), mappingDBQueries.get(0))
    .addMapping(normalizedDependencySchema.getPredicateByName("Exam"), mappingDBQueries.get(1))
    .addMapping(normalizedDependencySchema.getPredicateByName("Teaches"), mappingDBQueries.get(2))
    .addMapping(normalizedDependencySchema.getPredicateByName("Studies"), mappingDBQueries.get(3))
    .addMapping(normalizedDependencySchema.getPredicateByName("ExpertIn"), mappingDBQueries.get(4))
    .addMapping(normalizedDependencySchema.getPredicateByName("ComposesPlan"), mappingDBQueries.get(5))
    .build();

We now define a Conjunctive Query over the ontology

In [69]:
String queryString = """
    % Ontological Query
    (student) :- Exam(teacher, student, subject, data)
    """;
ConjunctiveQuery ontologicalQuery = (ConjunctiveQuery) queryParser.parse(queryString, normalizedDependencySchema.getAllPredicates()).get(0); //Provided by IMP-Logics

#### Rewritting the query

We have defined a new class Rewriter, using the metamodel of DependencySchema, that applies a FO-rewritting algorithm.
You can see the code of this class [here](https://github.com/inLabFIB/imp-logics-demo/blob/master/src/main/java/edu/upc/fib/inlab/imp/kse/ontological_queries_rewriting/Rewriter.java), and realize that it is using the concepts provided by IMP-Logics such as ConjunctiveQuery, TGD, MGUFinder, etc.

In [70]:
Set<TGD> ontologyTGDs = new HashSet<>(normalizedDependencySchema.getAllTGDs());
List<ConjunctiveQuery> rewriting = Rewriter.rewrite(ontologicalQuery, ontologyTGDs);

We can now print the query

In [71]:
QueryPrinter queryPrinter = new QueryPrinter();
for (int i = 0; i < rewriting.size(); i++) {
    Query queryToPrint = rewriting.get(i);
    printWithHeader("Query " + i, queryPrinter.print(queryToPrint));
}

Query 0: 
(student) :- Exam(teacher, student, subject, data)
Query 1: 
(student) :- Teaches(teacher, subject), Studies(student, subject)
Query 2: 
(student) :- AUX2(student, subject, teacher, data)
Query 3: 
(student) :- ExpertIn(teacher, subject), ComposesPlan(subject, unfTGD0), Studies(student, subject)
Query 4: 
(student) :- AUX1(student, subject, teacher)
Query 5: 
(student) :- HasPassed(student, subject)


#### Rewritting the query over the database

To finish the implementation of the query-rewritting, we need to translate the queries in terms of the database tables.
We have done so in the OBDA::translateToDBQueries functions, whose code is available [here](https://github.com/inLabFIB/imp-logics-demo/blob/master/src/main/java/edu/upc/fib/inlab/imp/kse/ontological_queries_rewriting/OBDAMapping.java#L38). Again, do note that it could
be implemented in few lines since most of the logic, that is, the unfolding, is already carried on IMP-Logics.

In [72]:
List<Query> finalRewriting = rewriting.stream()
    .map(mapping::translateToDBQueries)
    .flatMap(Collection::stream)
    .toList();

We can now print the query

In [73]:
for (int i = 0; i < finalRewriting.size(); i++) {
    Query queryToPrint = finalRewriting.get(i);
    printWithHeader("Query " + i, queryPrinter.print(queryToPrint));
}

Query 0: 
(student) :- DB_Exam(teacher, student, subject, data)
Query 1: 
(student) :- DB_Teaches(teacher, subject), DB_Studies(student, subject)
Query 2: 
(student) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2, DB_ComposesPlan(subject, unfTGD0), DB_Studies(student, subject)
Query 3: 
(student) :- DB_AcademicRecord(student, subject, mark), mark>5


#### We can use IMP-Logics asserts to check its validity

IMP-Logics is not only useful for developing the code, but also for checking its validity.
To validate the developed code, IMP-Logics also offers several testing facilities, such as the definition of several asserts.

In this example, we can check whether the 2nd query obtained query is isomorphic (i.e., the same up to variable-renaming) to an expected one:

In [74]:
QueryAssert.assertThat(finalRewriting.get(2))
    .isIsomorphicTo(List.of("st"), " DB_PublishesAbout(t, p, s), DB_PublishesAbout(t, p2, s), p<>p2, DB_ComposesPlan(s, sP), DB_Studies(st, s)")

edu.upc.fib.inlab.imp.kse.logics.logicschema.assertions.QueryAssert@1

Do note that the check fails if the actual query is not isomorphic to the expectation (here we change 'st' to 's' to make them non-isomorphic):

In [47]:
QueryAssert.assertThat(finalRewriting.get(2))
    .isIsomorphicTo(List.of("s"), " DB_PublishesAbout(t, p, s), DB_PublishesAbout(t, p2, s), p<>p2, DB_ComposesPlan(s, sP), DB_Studies(s, s)")

EvalException: [Actual query '(student) :- DB_PublishesAbout(teacher, paper, subject), DB_PublishesAbout(teacher, paper2, subject), paper<>paper2, DB_ComposesPlan(subject, unfTGD0), DB_Studies(student, subject)' is not isomorphic to
Expected query '(s) :- DB_PublishesAbout(t, p, s), DB_PublishesAbout(t, p2, s), p<>p2, DB_ComposesPlan(s, sP), DB_Studies(s, s)'] 
Expecting value to be true but was false